## Import

In [1]:
import sys
import math
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when, udf,isnan, count, sum, avg, format_number, round, to_timestamp, to_date, concat, lit

## Start Connection

In [2]:
spark = SparkSession.builder\
.master("local")\
.appName("Word Count")\
.getOrCreate()
#.config("spark.some.config.option", "some-value")\

## Basic Info

In [47]:
df1 = spark.read.options(header='true', inferSchema='true').csv("../data_source/chess_games.csv")

In [53]:
df1.show()
df1.printSchema()

+------------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+------------------+
|             Event|          White|          Black|Result|   UTCDate|            UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|                  AN|combined_timestamp|
+------------------+---------------+---------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+--------------------+------------------+
|        Classical |        eisaaaa|       HAMID449|   1-0|2016.06.30|2023-11-13 22:00:01|    1901|    1896|           11.0|          -11.0|D10|        Slav Defense|      300+5|Time forfeit|1. d4 d5 2. c4 c6...|              NULL|
|            Blitz |         go4jas|     Sergei1973|   0-1|2016.06.30|2023-1

### Check for row duplication

In [5]:
df1\
.groupBy(df1.columns)\
.count().\
where(col('count') > 1)\
.select(sum('count'))\
.withColumnRenamed("sum(count)", "dupliacte")\
.show()

+---------+
|dupliacte|
+---------+
|        2|
+---------+



### Check for NULL, NAN

In [6]:
columns_to_check = [c for c in df1.columns if c != 'UTCTime']

df1.select([count(when(col(c).isNull(), c)).alias(c) for c in columns_to_check]).show()

+-----+-----+-----+------+-------+--------+--------+---------------+---------------+---+-------+-----------+-----------+---+
|Event|White|Black|Result|UTCDate|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|Opening|TimeControl|Termination| AN|
+-----+-----+-----+------+-------+--------+--------+---------------+---------------+---+-------+-----------+-----------+---+
|    0|    0|    0|     0|      0|       0|       0|           4668|           4668|  0|      0|          0|          0|  0|
+-----+-----+-----+------+-------+--------+--------+---------------+---------------+---+-------+-----------+-----------+---+



In [7]:
columns_to_check = [c for c in df1.columns if c != 'UTCTime']

df1.select([count(when(isnan(c), c)).alias(c) for c in columns_to_check]).show()

+-----+-----+-----+------+-------+--------+--------+---------------+---------------+---+-------+-----------+-----------+---+
|Event|White|Black|Result|UTCDate|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|Opening|TimeControl|Termination| AN|
+-----+-----+-----+------+-------+--------+--------+---------------+---------------+---+-------+-----------+-----------+---+
|    0|    0|    0|     0|      0|       0|       0|              0|              0|  0|      0|          0|          0|  0|
+-----+-----+-----+------+-------+--------+--------+---------------+---------------+---+-------+-----------+-----------+---+



In [8]:
df1.filter(col("UTCTime").isNull()).count()

0

## Preprocessing

### Drop duplicate rows

In [9]:
df2 = df1.dropDuplicates().drop("AN")

## Format and filter an Event Column 

In [10]:
df3 = df2.filter(
    (df2["Event"].contains("Blitz")) |
    (df2["Event"].contains("Classic")) |
    (df2["Event"].contains("Bullet"))
)

In [11]:
df4 = df3.withColumn("Event", when(df3["Event"].contains("Blitz"), "Blitz").when(df3["Event"].contains("Classic"), "Classic").when(df3["Event"].contains("Bullet"), "Bullet").otherwise(df3["Event"]))

In [12]:
df5 = df4.filter(
    (df4["Result"] != '*') |
    (df4["WhiteRatingDiff"].isNotNull()) |
    (df4["BlackRatingDiff"].isNotNull())
)

In [13]:
df6 = df5.withColumn("Result", when((col("WhiteRatingDiff") < 0) & (col("BlackRatingDiff") > 0), "0-1")
    .when((col("WhiteRatingDiff") > 0) & (col("BlackRatingDiff") < 0), "1-0")
    .when((col("WhiteRatingDiff") == 0) & (col("BlackRatingDiff") == 0), "1/2-1/2")
    .otherwise(col("Result"))
)

In [14]:
mean_white_rating_diff_white_win = df6.filter(col("Result") == "1-0" ).agg(avg("WhiteRatingDiff")).collect()[0][0]
mean_black_rating_diff_white_win = df6.filter(col("Result") == "1-0" ).agg(avg("BlackRatingDiff")).collect()[0][0]
mean_white_rating_diff_black_win = df6.filter(col("Result") == "0-1" ).agg(avg("WhiteRatingDiff")).collect()[0][0]
mean_black_rating_diff_black_win = df6.filter(col("Result") == "0-1" ).agg(avg("BlackRatingDiff")).collect()[0][0]
mean_white_rating_diff_draw = df6.filter(col("Result") == "1/2-1/2" ).agg(avg("WhiteRatingDiff")).collect()[0][0]
mean_black_rating_diff_draw = df6.filter(col("Result") == "1/2-1/2" ).agg(avg("BlackRatingDiff")).collect()[0][0]

In [15]:
df7 = df6.withColumn("WhiteRatingDiff",
    when((col("Result") == "1-0") & col("WhiteRatingDiff").isNull(), mean_white_rating_diff_white_win)
    .when((col("Result") == "0-1") & col("WhiteRatingDiff").isNull(), mean_white_rating_diff_black_win)
    .when((col("Result") == "1/2-1/2") & col("WhiteRatingDiff").isNull(), mean_white_rating_diff_draw)
    .otherwise(col("WhiteRatingDiff"))
)
df8 = df7.withColumn("BlackRatingDiff",
    when((col("Result") == "1-0") & col("BlackRatingDiff").isNull(), mean_black_rating_diff_white_win)
    .when((col("Result") == "0-1") & col("BlackRatingDiff").isNull(), mean_black_rating_diff_black_win)
    .when((col("Result") == "1/2-1/2") & col("BlackRatingDiff").isNull(), mean_black_rating_diff_draw)
    .otherwise(col("BlackRatingDiff"))
)

In [16]:
df9 = df8.withColumn("WhiteRatingDiff",
    when((col("Result") == "1-0") & col("WhiteRatingDiff").isNull(), round(col("WhiteRatingDiff"), 1))
    .when((col("Result") == "0-1") & col("WhiteRatingDiff").isNull(), round(col("WhiteRatingDiff"), 1))
    .when((col("Result") == "1/2-1/2") & col("WhiteRatingDiff").isNull(), round(col("WhiteRatingDiff"), 1))
    .otherwise(col("WhiteRatingDiff"))
)
df10 = df9.withColumn("BlackRatingDiff",
    when((col("Result") == "1-0") & col("BlackRatingDiff").isNull(), round(col("BlackRatingDiff"), 1))
    .when((col("Result") == "0-1") & col("BlackRatingDiff").isNull(), round(col("BlackRatingDiff"), 1))
    .when((col("Result") == "1/2-1/2") & col("BlackRatingDiff").isNull(), round(col("BlackRatingDiff"), 1))
    .otherwise(col("BlackRatingDiff"))
)

In [23]:
df11 = df10.withColumn("UTCDate", to_date("UTCDate", "yyyy.MM.dd"))

In [58]:
df11 = df11.withColumn('WhiteRatingDiff', col('WhiteRatingDiff').cast(IntegerType()))
df11 = df11.withColumn('BlackRatingDiff', col('BlackRatingDiff').cast(IntegerType()))

df11.show()
df11.printSchema()

+-------+----------------+----------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+
|  Event|           White|           Black|Result|   UTCDate|            UTCTime|WhiteElo|BlackElo|WhiteRatingDiff|BlackRatingDiff|ECO|             Opening|TimeControl| Termination|
+-------+----------------+----------------+------+----------+-------------------+--------+--------+---------------+---------------+---+--------------------+-----------+------------+
| Bullet|   sefikzvekic12|         lazaeta|   1-0|2016-06-30|2023-11-13 22:00:54|    1816|    1671|              7|             -6|A04|Zukertort Opening...|      120+0|Time forfeit|
|  Blitz|       pokermoon|         definho|   0-1|2016-06-30|2023-11-13 22:02:44|    1694|    1855|             -7|              7|B00|        Owen Defense|      180+0|Time forfeit|
|  Blitz|        dondik96|      anylaury21|   1-0|2016-06-30|2023-11-13 22:03:49|    1755|

## Analytics

### Win-Loss Ratio: Calculate the win-loss ratio for White and Black players to understand if there is a bias toward one side winning more frequently.

In [27]:
df11.groupBy('Result').count().show()

+-------+-------+
| Result|  count|
+-------+-------+
|1/2-1/2|  47812|
|    1-0|3198560|
|    0-1|2985534|
+-------+-------+



### Popular Openings: Determine the most frequently played openings by examining the ECO codes

In [28]:
df11.groupBy('ECO').count().show()

+---+------+
|ECO| count|
+---+------+
|A23|   747|
|B05|  1311|
|B34|  8705|
|A47|  1299|
|E83|   597|
|E44|   168|
|D81|   114|
|B01|285707|
|C22| 21522|
|A65|  2852|
|E56|    47|
|A46| 33884|
|C78|  4449|
|B92|  2502|
|C24| 30051|
|C95|   282|
|C77|  5769|
|D16|   791|
|D98|    23|
|C98|    52|
+---+------+
only showing top 20 rows



### Analyze the success rates of these openings to identify which ones are more effective.

In [29]:
df11.groupBy("ECO").pivot("Result", values=["1-0", "0-1", "1/2-1/2"]).agg(count("*").alias("count")).show()

+---+------+------+-------+
|ECO|   1-0|   0-1|1/2-1/2|
+---+------+------+-------+
|A23|   387|   352|      8|
|B05|   622|   678|     11|
|E02|    16|    16|      1|
|B79|    12|    15|   NULL|
|D81|    57|    57|   NULL|
|E83|   297|   292|      8|
|B34|  4092|  4541|     72|
|A47|   643|   642|     14|
|E44|    98|    65|      5|
|A76|    35|    21|   NULL|
|B01|147777|135847|   2083|
|C22|  8833| 12553|    136|
|A65|  1446|  1388|     18|
|E56|    32|    15|   NULL|
|D28|     6|     9|   NULL|
|B92|  1177|  1307|     18|
|A46| 16954| 16625|    305|
|C78|  2559|  1865|     25|
|C24| 15630| 14199|    222|
|C95|   134|   146|      2|
+---+------+------+-------+
only showing top 20 rows



### Are certain openings associated with higher win rates for White or Black?

In [42]:
df12 = df11.groupBy("ECO", "Result").agg((when(col("Result") == "1-0", 1).otherwise(0)).alias("white_win"),
                                      (when(col("Result") == "0-1", 1).otherwise(0)).alias("black_win")) \
.groupBy("ECO").agg((sum("white_win") / (sum("white_win") + sum("black_win"))).alias("white_win_rate"),
                        (sum("black_win") / (sum("white_win") + sum("black_win"))).alias("black_win_rate"))

In [43]:
df12 = df12.withColumn("white_win_rate", format_number("white_win_rate", 2)) \
    .withColumn("black_win_rate", format_number("black_win_rate", 2)).show(n=df12.count(), truncate=False)

+---+--------------+--------------+
|ECO|white_win_rate|black_win_rate|
+---+--------------+--------------+
|A23|0.50          |0.50          |
|B05|0.50          |0.50          |
|B79|0.50          |0.50          |
|E02|0.50          |0.50          |
|D81|0.50          |0.50          |
|A47|0.50          |0.50          |
|E83|0.50          |0.50          |
|B34|0.50          |0.50          |
|A76|0.50          |0.50          |
|E44|0.50          |0.50          |
|B01|0.50          |0.50          |
|C22|0.50          |0.50          |
|A65|0.50          |0.50          |
|D28|0.50          |0.50          |
|E56|0.50          |0.50          |
|B92|0.50          |0.50          |
|A46|0.50          |0.50          |
|C78|0.50          |0.50          |
|C24|0.50          |0.50          |
|C95|0.50          |0.50          |
|C98|0.50          |0.50          |
|C77|0.50          |0.50          |
|D16|0.50          |0.50          |
|D98|0.50          |0.50          |
|A92|0.50          |0.50    

### Do shorter time controls lead to more decisive results, such as wins or losses?

### Do longer games tend to favor stronger players?

### Track the popularity of openings over time and see if certain openings go in and out of fashion

In [ ]:
df2.groupBy('Event').count().show()

In [ ]:
df2.groupBy('Result').count().show()

In [ ]:
df2.groupBy('Termination').count().show()

In [ ]:
df1.where("Termination = 'Rules infraction'").show()

In [ ]:
df1.where("Termination = 'Abandoned'").show()

In [ ]:
df2.where("Result = '*'").show()

In [ ]:
df2.where("Result = '*'").groupBy('Termination').count().show()

In [ ]:
df2.show()

In [ ]:
columns_to_check = [c for c in df1.columns if c != 'UTCTime']

df2.select([count(when(col(c).isNull(), c)).alias(c) for c in columns_to_check]).show()

In [ ]:
df5.where("Result = '*'").show()

In [ ]:
df8.filter(col("WhiteRatingDiff").isNull()).show()

In [ ]:
df6.filter(col("WhiteRatingDiff").isNull() & col("BlackRatingDiff").isNull() & (col("Result") != '*')).show()

In [ ]:
df5.filter(col("WhiteRatingDiff").isNotNull() & col("BlackRatingDiff").isNotNull() & (col("Result") == '*')).show()

In [ ]:
columns_to_check = [c for c in df11.columns if c != 'UTCTime']

df11.select([count(when(col(c).isNull(), c)).alias(c) for c in columns_to_check]).show()